In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np

# Set random seed for reproducibility
torch.manual_seed(42)

# Custom Dataset class for CSV files
class CSVDataset(Dataset):
    def __init__(self, csv_file):
        self.data_frame = pd.read_csv(csv_file)
        self.labels = self.data_frame.iloc[:, -1]
        self.images = self.data_frame.iloc[:, :-1]

        # Create a label mapping if the labels are not numeric
        self.label_mapping = {label: idx for idx, label in enumerate(sorted(self.labels.unique()))}
        # Convert all labels using the mapping
        self.labels = self.labels.map(self.label_mapping)

        print(f"Unique labels: {self.labels.unique()}")
        assert self.labels.max() < 11, "Label out of bounds"  # Update '10' to your number of classes

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        label = self.labels.iloc[idx]
        image = self.images.iloc[idx].values.astype(np.float32).reshape(1, 32, 64)  # Adjust as per your data
        return torch.tensor(image), torch.tensor(label)


# Load your datasets
train_dataset = CSVDataset('C:\\Users\\fhitl\\Documents\\Deep_Neural_Networks_Assignments\\datasets\\food\\food11-rn50-validation.csv')
validation_dataset = CSVDataset('C:\\Users\\fhitl\Documents\\Deep_Neural_Networks_Assignments\\datasets\\food\\food11-rn50-training.csv')
evaluation_dataset = CSVDataset('C:\\Users\\fhitl\Documents\\Deep_Neural_Networks_Assignments\\datasets\\food\\food11-rn50-evaluation.csv')

train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)
validation_loader = DataLoader(validation_dataset, batch_size=4, shuffle=False)
evaluation_loader = DataLoader(evaluation_dataset, batch_size=4, shuffle=False)

# Convolutional Neural Network with Dropout
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.dropout = nn.Dropout(0.5)
        self.fc1 = nn.Linear(16 * 5 * 13, 120)  # Adjusted size
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 11)  # Adjust for number of classes

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 13)  # Adjust the flattened size
        x = self.dropout(x)  # Apply dropout after flattening
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Instantiate the model, loss function, and optimizer
model = ConvNet().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Learning rate scheduler
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

# Training loop with early stopping
num_epochs = 26
best_val_loss = float('inf')
early_stopping_counter = 0

for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    # Validation
    model.eval()
    with torch.no_grad():
        val_loss = 0
        correct = 0
        total = 0
        for images, labels in validation_loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    # Learning rate scheduler step
    scheduler.step()

    # Early stopping
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        early_stopping_counter = 0
    else:
        early_stopping_counter += 1
        if early_stopping_counter > 2:  # Stop if no improvement after 2 epochs
            print("Early stopping triggered")
            break

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss / len(train_loader)}, Validation Loss: {val_loss / len(validation_loader)}, Validation Accuracy: {100 * correct / total}%')

# Evaluate on the Evaluation Dataset
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in evaluation_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    print(f'Accuracy of the network on the evaluation dataset: {100 * correct / total}%')

# Save the model
PATH = './cnn.pth'
torch.save(model.state_dict(), PATH)